------------------------------- *IMPORTING REQUIRED LIBRARIES* -------------------------------

In [ ]:
import numpy as np
import pandas as pd
import re
import spacy
import time
import json
from tqdm import tqdm

In [ ]:
nlp = spacy.load('en_core_web_sm')

------------------------------- *IMPORTING DATASET* -------------------------------

In [ ]:
csv_dataset = pd.read_csv("dataset_combined/combined_dataset_csv.csv")
dataset = csv_dataset.iloc[400:1000000,1:].values

In [ ]:
# dataset[780000]

In [ ]:
prev = []
my_list = []
q_a = {'question':'answer'}
question = []
answer = []
q = ""
a = ""
previd=dataset[0][0]
start=0
i = int(0)

row_count = dataset.shape[0]

------------------------------- *SEGREGATION INTO QUESTIONS AND ANSWERS* -------------------------------

In [ ]:
for x in tqdm(dataset):
    if i < row_count and previd != dataset[i][0]:
        for j in range(start,i):
            from1 = dataset[j][2]
            if (j==start) and dataset[j][3] == dataset[j][3]:
                break
            while (j < row_count) and (dataset[j][3] != dataset[j][3] or dataset[j][3] != from1):
                q = q + str(dataset[j][4])
                j = j+1

            while j < row_count and dataset[j][3] == from1 :#tofrom
                if a == "":
                    a = str(dataset[j][4])
                else:
                    a = a + "\n" + str(dataset[j][4])
                j=j+1
                
            if(a != ""):
                answer.append(a)

            if(a != "" and q != ""):
                question.append(q)
            a=""
            q=""
            if j >= i:
                break
        previd = dataset[i][0]
        start = i
    i=i+1

------------------------------- *NATURAL LANGUAGE PROCESSING TO CLEAN QUESTIONS* -------------------------------

In [ ]:
# function using regex to remove contractions in the questions
def clean_text(text):
    text = text.lower()
    text = re.sub(r"i'm", "i am", text)
    text = re.sub(r"he's", "he is", text)
    text = re.sub(r"she's", "she is", text)
    text = re.sub(r"that's", "that is", text)
    text = re.sub(r"what's", "what is", text)
    text = re.sub(r"where's", "where is", text)
    text = re.sub(r"\'ll", " will", text)
    text = re.sub(r"\'ve", " have", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"\'d", " would", text)
    text = re.sub(r"won't", "will not", text)
    text = re.sub(r"can't", "cannot", text)
    return text

In [ ]:
# function to convert list to a string
def listToString(s):
    str1 = ""  
    for ele in s:  
        str1 += ele  
    return str1  

In [ ]:
# removing STOP-WORDS from questions
for i in range(len(question)):
    ques = []
    temp = clean_text(question[i].lower())
    for token in temp.split(' '):
        t = str(token)
        if nlp.vocab[t].is_stop:
            pass
        else:
            ques.append(t)
    ques_list = ' '.join(ques)
    question[i] = listToString(ques_list)

In [ ]:
# lemmatization of questions
for i in range(len(question)):
    ques2 = []
    temp2 = nlp(question[i])
    for token2 in temp2:
        
        ques2.append(token2.lemma_)
    ques_list = ' '.join(ques2)
    question[i] = listToString(ques_list)

In [ ]:
# exporting questions and answers as json format to be fed to the model
with open('data10lac/data.json', 'w') as outfile:
    x = json.dump([{'message': ques, 'response': ans} for ques, ans in zip(question, answer)], outfile)